In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.10.0 

In [36]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import urllib.request
import requests
#from bs4 import BeautifulSoup
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Geocoder to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
import folium 

print("Importing libraries complete!")

Importing libraries complete!


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Table"
table_list = pd.read_html(url)
#print("number of tables in web page = ", len(table_list))
Raw_df = table_list[11] # what we want is the 11th table in the list

Raw_df.head()

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN


In [4]:
Raw_df.drop(columns=['Map', 'Unnamed: 5'], axis=1, inplace = True)
#DataFrame.drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise'
#Raw_df.sort_values("CDN number", axis=0, ascending=True, inplace=True)

In [5]:
Neighbour_df = Raw_df.rename(columns={'CDN number': 'CDN', 
                                     'City-designated neighbourhood':'Neighbourhood_Name', 
                                     'Former city/borough':'Borough',
                                     'Neighbourhoods covered':'Areas_Covered'
                                    })

Neighbour_df.head()

,CDN,Neighbourhood_Name,Borough,Areas_Covered
0,129,Agincourt North,Scarborough,Agincourt and Brimwood
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern
2,20,Alderwood,Etobicoke,Alderwood
3,95,Annex,Old City of Toronto,The Annex and Seaton Village
4,42,Banbury-Don Mills,North York,Don Mills


In [6]:
# Explode/Split Neighbourhood column into multiple rows (removing the hyphen symbol)
new_df = pd.DataFrame(Neighbour_df.Neighbourhood_Name.str.split('-').tolist(), index=Neighbour_df.CDN).stack()
new_df = new_df.reset_index([0, 'CDN'])
new_df['Latitude'] = np.NaN
new_df['Longitude'] = np.NaN
new_df.columns = ['CDN', 'Neighbourhood', 'Latitude', 'Longitude']
new_df.head(10)

,CDN,Neighbourhood,Latitude,Longitude
0,129,Agincourt North,NaN,NaN
1,128,Agincourt South,NaN,NaN
2,128,Malvern West,NaN,NaN
3,20,Alderwood,NaN,NaN
4,95,Annex,NaN,NaN
5,42,Banbury,NaN,NaN
6,42,Don Mills,NaN,NaN
7,34,Bathurst Manor,NaN,NaN
8,76,Bay Street Corridor,NaN,NaN
9,52,Bayview Village,NaN,NaN


In [7]:
#geolocator = Nominatim(user_agent="Sample_Data")
#location = geolocator.geocode("Humbermede Toronto Ontario")
#print(location.raw)


from functools import partial
#rom geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="Neighbourhood_explorer")

_geocode = partial(geolocator.geocode, language="en")
geocode = lambda query, **kw: _geocode("%s, Toronto Ontario Canada" % query, **kw)

#geocode = lambda query: geolocator.geocode("%s, ON Canada" % query)
#geocode = RateLimiter(geocode_, min_delay_seconds=1)

def eval_lat(x):
    try:
        #print(x)
        return (x.latitude)
    except:
        return (None)
def eval_lon(x):
    try:
        return (x.longitude)
    except:
        return (None)

    
new_df['Latitude'] = new_df['Neighbourhood'].apply(geocode).apply(lambda x: eval_lat(x))
new_df['Longitude'] = new_df['Neighbourhood'].apply(geocode).apply(lambda x: eval_lon(x))

print("Geo Coding complete!")



Geo Coding complete!


In [8]:
#new_df.to_csv('TorontoNeighbourhoodGeocoding.csv', index = False)

In [7]:
#Import the geocode data from csv file 
new_df = pd.read_csv("TorontoNeighbourhoodGeocoding.csv")

In [8]:
new_df

,CDN,Neighbourhood,Latitude,Longitude
0,129,Agincourt North,43.808038,-79.266439
1,128,Agincourt South,43.785353,-79.278549
2,128,Malvern West,43.790679,-79.248491
3,20,Alderwood,43.601717,-79.545232
4,95,Annex,43.670338,-79.407117
5,42,Banbury,43.742796,-79.369957
6,42,Don Mills,43.775347,-79.345944
7,34,Bathurst Manor,43.665519,-79.411937
8,76,Bay Street Corridor,43.667342,-79.388457
9,52,Bayview Village,43.769197,-79.376662


In [9]:
df1 = new_df[new_df.isna().any(axis=1)]
df1

,CDN,Neighbourhood,Latitude,Longitude
17,122,Birchcliffe,NaN,NaN
40,33,Clanton Park,NaN,NaN
56,26,CFB,NaN,NaN
88,22,Humbermede,NaN,NaN
93,14,City Centre West,NaN,NaN
104,114,Lambton Baby Point,NaN,NaN
143,67,Playter Estates,NaN,NaN
172,3,Beaumond Heights,NaN,NaN
182,1,Clairville,NaN,NaN
187,91,Pellam Park,NaN,NaN


In [10]:
new_df.dropna(axis=0, inplace=True)
print("shape before removing duplicates: ",new_df.shape)
new_df.drop_duplicates(subset ="CDN", keep = 'first', inplace = True) 
print("shape after removing duplicates: ",new_df.shape)

shape before removing duplicates:  (195, 4)
shape after removing duplicates:  (136, 4)


In [12]:

address = 'Casa Loma,ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Casa Loma,ON are 43.6781015, -79.409415775.


In [13]:
# create map of Toronto using latitude and longitude values created above
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [14]:
Toronto_df = pd.merge(Neighbour_df, new_df, how = 'inner', on = "CDN")
Toronto_df.head(10)

,CDN,Neighbourhood_Name,Borough,Areas_Covered,Neighbourhood,Latitude,Longitude
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,Agincourt North,43.808038,-79.266439
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,Agincourt South,43.785353,-79.278549
2,20,Alderwood,Etobicoke,Alderwood,Alderwood,43.601717,-79.545232
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,Annex,43.670338,-79.407117
4,42,Banbury-Don Mills,North York,Don Mills,Banbury,43.742796,-79.369957
5,34,Bathurst Manor,North York,Bathurst Manor,Bathurst Manor,43.665519,-79.411937
6,76,Bay Street Corridor,Old City of Toronto,"Bay Street, Financial District",Bay Street Corridor,43.667342,-79.388457
7,52,Bayview Village,North York,Bayview Village,Bayview Village,43.769197,-79.376662
8,49,Bayview Woods-Steeles,North York,Bayview Woods,Bayview Woods,43.798127,-79.382973
9,39,Bedford Park-Nortown,North York,"Bedford Park, Ledbury Park, and Nortown",Bedford Park,43.737388,-79.410925


Get the Asian Population in each neighbourhood

In [20]:
SouthAsian_df = pd.read_csv('SoutAsianPopulation_Toronto.csv')
SouthAsian_df.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,0,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,1583,Ethnic origin,Ethnic origin population,Census Profile 98-316-X2016001,South Asian origins,350040,5260,4170,575,1320,2940,565,2515,1410,805,535,410,8225,1530,2320,525,445,410,655,1245,525,275,355,2525,2730,6060,495,2995,390,760,1110,2945,7380,2390,1685,750,1760,400,6440,870,470,1340,1220,7460,265,280,2600,1125,970,3295,1055,910,4685,1520,415,3520,3320,355,3845,3595,535,985,4360,815,3670,275,315,7265,1135,430,425,520,480,380,18085,1025,255,3215,1940,5370,1400,690,11395,785,2035,645,795,1120,2420,495,3800,4750,800,1775,370,455,3685,765,275,1330,615,2780,1440,1155,800,685,20265,305,540,5535,1700,1240,1205,1580,1010,4750,5850,795,2770,9545,290,440,3070,7130,5420,14275,815,885,470,4245,3460,935,2455,21505,680,355,330,470,620,4455,970
2,1598,Ethnic origin,Ethnic origin population,Census Profile 98-316-X2016001,South Asian origins; n.i.e.,18470,225,185,10,70,80,15,90,120,25,30,15,485,35,55,70,30,25,45,80,0,0,0,7

In [21]:
#DataFrame.drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise'
SouthAsian_df = SouthAsian_df.drop(columns=["_id","Category","Topic","Data Source","Characteristic"], axis=1)

SouthAsian_df.head()

,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,0,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,350040,5260,4170,575,1320,2940,565,2515,1410,805,535,410,8225,1530,2320,525,445,410,655,1245,525,275,355,2525,2730,6060,495,2995,390,760,1110,2945,7380,2390,1685,750,1760,400,6440,870,470,1340,1220,7460,265,280,2600,1125,970,3295,1055,910,4685,1520,415,3520,3320,355,3845,3595,535,985,4360,815,3670,275,315,7265,1135,430,425,520,480,380,18085,1025,255,3215,1940,5370,1400,690,11395,785,2035,645,795,1120,2420,495,3800,4750,800,1775,370,455,3685,765,275,1330,615,2780,1440,1155,800,685,20265,305,540,5535,1700,1240,1205,1580,1010,4750,5850,795,2770,9545,290,440,3070,7130,5420,14275,815,885,470,4245,3460,935,2455,21505,680,355,330,470,620,4455,970
2,18470,225,185,10,70,80,15,90,120,25,30,15,485,35,55,70,30,25,45,80,0,0,0,75,145,455,25,215,10,20,95,75,400,150,60,30,100,20,310,35,25,30,120,455,25,10,90,20,40,310,10,35,195,65,35,115,155,25,165,220,20,50,330,45,275,0,0,230,70,0,30,0,25,0,870,40,10,100,75,180,60,50,455,20,60,15,50,45,85,25,360,555,20,180,55,25,250,35,10,45,20,150,115,35,45,0,700,0,0,405,135,75,50,55,10,335,570,50,90,860,15,10,190,335,305,5

In [22]:
#SouthAsian_df
#SouthAsian_df.iloc[0,0] = 0
SouthAsian_df.astype('int64')
#SouthAsian_df.dtypes

,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,0,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,350040,5260,4170,575,1320,2940,565,2515,1410,805,535,410,8225,1530,2320,525,445,410,655,1245,525,275,355,2525,2730,6060,495,2995,390,760,1110,2945,7380,2390,1685,750,1760,400,6440,870,470,1340,1220,7460,265,280,2600,1125,970,3295,1055,910,4685,1520,415,3520,3320,355,3845,3595,535,985,4360,815,3670,275,315,7265,1135,430,425,520,480,380,18085,1025,255,3215,1940,5370,1400,690,11395,785,2035,645,795,1120,2420,495,3800,4750,800,1775,370,455,3685,765,275,1330,615,2780,1440,1155,800,685,20265,305,540,5535,1700,1240,1205,1580,1010,4750,5850,795,2770,9545,290,440,3070,7130,5420,14275,815,885,470,4245,3460,935,2455,21505,680,355,330,470,620,4455,970
2,18470,225,185,10,70,80,15,90,120,25,30,15,485,35,55,70,30,25,45,80,0,0,0,75,145,455,25,215,10,20,95,75,400,150,60,30,100,20,310,35,25,30,120,455,25,10,90,20,40,310,10,35,195,65,35,115,155,25,165,220,20,50,330,45,275,0,0,230,70,0,30,0,25,0,870,40,10,100,75,180,60,50,455,20,60,15,50,45,85,25,360,555,20,180,55,25,250,35,10,45,20,150,115,35,45,0,700,0,0,405,135,75,50,55,10,335,570,50,90,860,15,10,190,335,305,5

fsdf
THis is a text

In [30]:
SouthAsian_df.append(SouthAsian_df.sum(),ignore_index=True)

,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,0,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,350040,5260,4170,575,1320,2940,565,2515,1410,805,535,410,8225,1530,2320,525,445,410,655,1245,525,275,355,2525,2730,6060,495,2995,390,760,1110,2945,7380,2390,1685,750,1760,400,6440,870,470,1340,1220,7460,265,280,2600,1125,970,3295,1055,910,4685,1520,415,3520,3320,355,3845,3595,535,985,4360,815,3670,275,315,7265,1135,430,425,520,480,380,18085,1025,255,3215,1940,5370,1400,690,11395,785,2035,645,795,1120,2420,495,3800,4750,800,1775,370,455,3685,765,275,1330,615,2780,1440,1155,800,685,20265,305,540,5535,1700,1240,1205,1580,1010,4750,5850,795,2770,9545,290,440,3070,7130,5420,14275,815,885,470,4245,3460,935,2455,21505,680,355,330,470,620,4455,970
2,18470,225,185,10,70,80,15,90,120,25,30,15,485,35,55,70,30,25,45,80,0,0,0,75,145,455,25,215,10,20,95,75,400,150,60,30,100,20,310,35,25,30,120,455,25,10,90,20,40,310,10,35,195,65,35,115,155,25,165,220,20,50,330,45,275,0,0,230,70,0,30,0,25,0,870,40,10,100,75,180,60,50,455,20,60,15,50,45,85,25,360,555,20,180,55,25,250,35,10,45,20,150,115,35,45,0,700,0,0,405,135,75,50,55,10,335,570,50,90,860,15,10,190,335,305,5

In [154]:
## Defining Foursquare Credentials 
CLIENT_ID = 'HH15X12VLCIU2RJJ0233AP50Y4ZZ2ED1FNG5AHVAAK1FNUXB' # your Foursquare ID
CLIENT_SECRET = '0U2TM5EKCMVP1S11YS0XJBHNW5H2BMKMFDSVAVYAY0FQMHSD' # your Foursquare Secret
VERSION = '20200530' # Foursquare API version

#https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT

# Define the radius and limit of reponse from foursquare 
RADIUS = 500
LIMIT = 100
CATEGORYID = '5413605de4b0ae91d18581a9,4bf58dd8d48988d10f941735,4bf58dd8d48988d142941735' #pass as 'categoryId' 

#Sri Lankan Restaurant - 5413605de4b0ae91d18581a9 (covers only Sri Lankan resturants)
#Indian Restaurants - 4bf58dd8d48988d10f941735 (this category covers multiple indian cusines)
#Asian Restaurants - 4bf58dd8d48988d142941735 (this category covers chineese,Japaneese and south east asian cusines)
#parent food category - 4d4b7105d754a06374d81259


#For Temporary checking lat,lng set to toronto city
lat = 43.706740 
lng = -79.398328


In [156]:
#venuecategories = requests.get('https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)).json()
#venuecategories["response"]['categories'][3] #these are the parent food category

In [157]:
# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    CATEGORYID,
    RADIUS, 
    LIMIT)

# make the GET request
results = requests.get(url).json()["response"]['groups'][0]['items']

In [158]:
#results

In [159]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          lat, 
          lng, 
          CATEGORYID,
          RADIUS, 
          LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [160]:
Toronto_venues = getNearbyVenues(names= Toronto_df['Neighbourhood_Name'],
                                 latitudes= Toronto_df['Latitude'],
                                 longitudes= Toronto_df['Longitude'],
                                 radius = RADIUS,
                                 limit = LIMIT
                                )


print(Toronto_venues.shape)
Toronto_venues.head(7)



(1034, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
1,Agincourt North,43.808038,-79.266439,Congee Town 太皇名粥,43.809035,-79.267634,Chinese Restaurant
2,Agincourt North,43.808038,-79.266439,Kin Kin Bubble Tea Co,43.807852,-79.270296,Chinese Restaurant
3,Agincourt North,43.808038,-79.266439,Bento Box,43.809248,-79.269029,Japanese Restaurant
4,Agincourt North,43.808038,-79.266439,Xe Lua Vietnamese Cuisine 火車頭,43.809224,-79.269547,Vietnamese Restaurant
5,Agincourt North,43.808038,-79.266439,Casa Deluz 豪門宴,43.808955,-79.268113,Dim Sum Restaurant
6,Agincourt South-Malvern West,43.785353,-79.278549,One2 Snacks,43.787048,-79.276658,Asian Restaurant


In [161]:
#Toronto_venues

In [163]:
#Toronto_venues.groupby('Neighborhood').count()
#Toronto_venues.head(10)
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 47 uniques categories.


In [164]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']
Toronto_onehot.head()

,Asian Restaurant,Bakery,Bar,Breakfast Spot,Bubble Tea Shop,Buffet,Cambodian Restaurant,Cantonese Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Dessert Shop,Dim Sum Restaurant,Dongbei Restaurant,Dumpling Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Truck,Fried Chicken Joint,Grocery Store,Hong Kong Restaurant,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Malay Restaurant,Mexican Restaurant,Noodle House,North Indian Restaurant,Peking Duck Restaurant,Ramen Restaurant,Restaurant,Sake Bar,Sandwich Place,South Indian Restaurant,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Tibetan Restaurant,Udon Restaurant,Vietnamese Restaurant,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agincourt North
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agincourt North
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agincourt North
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agincourt North
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Agincourt North


In [167]:
print(Toronto_onehot.shape)
#Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
#Toronto_grouped.head(10)

#Toronto_grouped.shape

(1034, 48)
